# Solving game

The task of this notebook is to implement the game of solving ASP programs!

This should be done by a propagator that: 
* profiles the solving process, and
* allows the user to make the solving decisions during search

You can start from the script `solving-game.py`
and follow the instructions attached to the comments.

Before you start, we recommend you to read sections 6 and 7 of [1].

[1] [Kaminski, R., Romero, J., Schaub, T., & Wanko, P. (2023). How to Build Your Own ASP-based System?! Theory Pract. Log. Program., 23(1), 299–361.](https://www.cs.uni-potsdam.de/wv/publications/DBLP_journals/tplp/KaminskiRSW23.pdf)

We start with a simple example:

In [1]:
! cat example.lp

dom(1..2).
{ a(X) } :- dom(X).
{ b(X) } :- dom(X).
:- a(X), not b(X).
:- not a(X), b(X).

The output should look like this (we used our final script to generate the next cell):

In [ ]:
! python my_solving_game.py example.lp

clingo version 5.7.1
Reading from example.lp
symbol		is_fact		is_external	program_literal	solver_literal
------------------------------------------------------------------------------
dom(1)		True		False		1		1
dom(2)		True		False		2		1
a(1)		False		False		7		4
a(2)		False		False		8		5
b(1)		False		False		3		2
b(2)		False		False		4		3
c(1)		False		False		5		2
c(2)		False		False		6		3

Assignment: 
  decision_level: 0
  has_conflict: False
  is_total: False
  literals in the assignment: 1 2 3 4 5
  trail: 1

0:dom(1)_dom(2) (P)
Solving...
b(1)_c(1) b(2)_c(2) a(1) a(2) ? 

The script is interactive, and in the example we have chosen at the first decision point the atom `a(1)`, and in the second one `a(2)`. After that, propagation leads to an answer set.

The following command should lead to a similar result:

In [ ]:
! python solving-agent.py example.lp